In [1]:
import torch
import matplotlib.pyplot as plt
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.transforms.functional import InterpolationMode
from utils import DiffusionModelPipeline, ImagePromptDataset

c:\Users\wolfg\OneDrive\Documents\Projects\OMI\my-omi-code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dit_params = {
    'channels': 384,
    'nBlocks': 8,
    'inC': 16, 
    'nHeads': 8,
    'patchSize': 2
}

cat_params = {
    'input_dim': 768,  # T5-base hidden size
    'hidden_dim': 512,
    'output_dim': 256,
    'num_layers': 3,
    'num_heads': 8
}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DiffusionModelPipeline(dit_params = dit_params,
                       cat_params = cat_params,
                       device=device)

In [4]:
##Hacky work around - will fix later
model.tokenizer.name_or_path = 't5-base'
model.text_encoder.config.name_or_path = 't5-base'
model.vae.config._name_or_path = 'flux_schnell_vae'

## Test Gen

In [5]:
# image = model('a beautiful sunset over the ocean', num_inference_steps=1000)


In [6]:
# image[0]

## Test Training

In [7]:
# dataset = load_dataset("openmodelinitiative/initial-test-dataset")
# dataset = load_dataset("laion/laion2B-en")
ds = load_dataset("svjack/pokemon-blip-captions-en-zh")
split_ds = ds['train'].train_test_split(test_size=0.1, seed=42)


In [8]:
split_ds

DatasetDict({
    train: Dataset({
        features: ['image', 'en_text', 'zh_text'],
        num_rows: 749
    })
    test: Dataset({
        features: ['image', 'en_text', 'zh_text'],
        num_rows: 84
    })
})

In [10]:
transform = transforms.Compose([
    transforms.Resize((128, 128), interpolation=InterpolationMode.LANCZOS),
    transforms.ToTensor(),  # Converts to tensor and scales pixel values to [0, 1]
])

In [11]:
# Create the custom dataset
train_dataset = ImagePromptDataset(split_ds["train"], transform=transform)
val_dataset = ImagePromptDataset(split_ds["test"], transform=transform)


In [12]:
# Training DataLoader
train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,  # Adjust based on your GPU memory
    shuffle=True,
    num_workers=0  # Set to 0 if you encounter issues on Windows
)

# Validation DataLoader
val_dataloader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0
)

In [12]:
model.train(train_dataloader,
            val_dataloader,
            epochs = 10000,
            log_interval= 100,
            save_interval= 100,
            output_dir= 'pokemon_training_test')

changing optimizer to AdamW and added rectified flow and CaT


Epoch 1:   0%|          | 0/188 [00:00<?, ?it/s]c:\Users\wolfg\OneDrive\Documents\Projects\OMI\my-omi-code\.venv\Lib\site-packages\diffusers\models\attention_processor.py:2358: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  hidden_states = F.scaled_dot_product_attention(


Model saved to pokemon_training_test\best_model


Model saved to pokemon_training_test\best_model


Model saved to pokemon_training_test\best_model


Model saved to pokemon_training_test\best_model


In [ ]:
# model.train(train_dataloader,
#             val_dataloader,
#             epochs = 2,
#             log_interval= 100,
#             save_interval= 2,
#             output_dir= 'pokemon_training_test')

In [ ]:
img1 = model.generate('A green pokemon with a leaf on its head',
               num_inference_steps=1000)
img1[0]

In [ ]:
img2 = model.generate('A green pokemon with a leaf on its head',
               num_inference_steps=50)
img2[0]